<a href="https://colab.research.google.com/github/dini-5002/Grammar_check/blob/main/FineTuned1_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing required modules

In [ ]:
!pip install datasets

In [ ]:
!pip install datasets

In [ ]:
 !pip install dataset transformers==4.28.0

In [ ]:
!pip install wandb

In [ ]:
!pip install transformers[torch]

##Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
from transformers import (AdamW,
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )


#Loading model and finetuning on the required dataset

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')
model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
df = pd.read_csv('/content/C4200M_50k.csv')
df.head()

,Unnamed: 0,input,label
0,0,The effect of widespread dud targets two face ...,"1. The effect of ""widespread dud"" targets two ..."
1,1,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
2,2,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
3,3,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
4,4,"Fairy Or Not, I'm the Godmother: no just look,...","Fairy Or Not, I'm the Godmother: Not just a lo..."


In [ ]:
#split it into training and evaluation
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(eval_df)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer, print_text= False, trucation = True):
        self.dataset = dataset
        self.pad_to_max_length = True
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)

    def tokenize_data(self, example):
      input_, target_ = example['input'], example['label']

      #tokenize inputs
      tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, max_length = self.max_len,
                                   return_attention_mask=True, truncation=True)
      tokenized_targets = tokenizer(target_, pad_to_max_length = self.pad_to_max_length, max_length=self.max_len, return_attention_mask=True,truncation=True)
      inputs = {"input_ids": tokenized_inputs['input_ids'],
                "attention_mask": tokenized_inputs['attention_mask'],
                "labels": tokenized_targets['input_ids']}
      return inputs

    def  __getitem__(self, index):
       inputs = self.tokenize_data(self.dataset[index])
       return inputs

In [ ]:
datacollater = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
args = Seq2SeqTrainingArguments(output_dir='./results',
                                evaluation_strategy="epoch",
                                learning_rate=2e-5,
                                per_device_train_batch_size=16,
                                per_device_eval_batch_size=16,
                                fp16=True,
)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    tokenizer = tokenizer,
    train_dataset=GrammarDataset(train_dataset, tokenizer),
    eval_dataset=GrammarDataset(test_dataset, tokenizer),
    data_collator=datacollater,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.425200,0.380454
2,0.405700,0.369074


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/modelT5f4.pt')

In [ ]:
mod=T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')

In [ ]:
mod.load_state_dict(torch.load('/content/gdrive/MyDrive/modelT5f4.pt'))

In [ ]:
def correct_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    output = mod.generate(**inputs)
    corrected_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return corrected_text


In [ ]:
sentence = "the boy look here"

In [ ]:
correct_text(sentence)